In [44]:
import pandas as pd
import time
import sys
from pathlib import Path

# Go up one directory to get to master/
project_root = str(Path.cwd().parent)
sys.path.append(project_root)



In [45]:
depth_result = 8
name_dataset = "adult"

Achtung: Man muss noch in den Ergebnissen zuordnen was in den ursprungsdatensatz (das binarized wurde) als positive und was als negative gesehen wird. Also wie das label jetzt heißt dem ursprünglichen label zuordnen

In [46]:

# remember to modify path
def get_solution_pulp(dataset: str, depth: int) -> pd.DataFrame:
    solution = pd.read_csv(f"../results/{dataset}/output_depth_{depth}_classification_test_pulp.csv")
    target_labels_col = solution['y']
    prediction_col = solution['prediction']
    return solution, target_labels_col, prediction_col

In [47]:
solution, target_labels_col, prediction_col  = get_solution_pulp(dataset = name_dataset, depth=depth_result)

damit das hier immer funktioniert, muss das mapping vom binarizer auf die Zahlen immer gleich sein

In [48]:
# specifics can of course vary (maybe use list or dict later)

origin_dataset_test = pd.read_csv(f"../datasets/{name_dataset}/adult.test", sep=',', skipinitialspace=True, header=None) #16281 rows
# Remove dots from the 'target' column
origin_target_col = origin_dataset_test[14].astype(str).str.replace('.', '', regex=False)
#print(origin_target_col)

df = pd.concat([origin_target_col, target_labels_col], axis=1)

mapping = df.drop_duplicates().set_index('y')[14].to_dict() #this should fail if columns and the binarization dont consistently match
print(mapping)

#Convention for now: >50K is positive, <=50K is negative

{1: '<=50K', 2: '>50K'}


In [49]:
print(target_labels_col)
print(prediction_col)

0        1
1        1
2        2
3        2
4        1
        ..
16276    1
16277    1
16278    1
16279    1
16280    2
Name: y, Length: 16281, dtype: int64
0        1
1        1
2        1
3        2
4        1
        ..
16276    1
16277    1
16278    2
16279    1
16280    2
Name: prediction, Length: 16281, dtype: int64


In [ ]:
def acc_sens_spec_prec(solution, pos_represent, neg_represent): #16281
    tp = len(solution[(solution['y'] == pos_represent) & (solution['prediction'] == pos_represent)])
    tn = len(solution[(solution['y'] == neg_represent) & (solution['prediction'] == neg_represent)])
    fp = len(solution[(solution['y'] == neg_represent) & (solution['prediction'] == pos_represent)])
    fn = len(solution[(solution['y'] == pos_represent) & (solution['prediction'] == neg_represent)])
    number_datapoints = tp+tn+fp+fn
    if number_datapoints != len(solution):
        print('error in calculation of acc')
    sensitivity = tp/(tp+fn) #also called recall, True Positive Rate
    specificity = tn/(tn+fp)
    precision = tp/(tp+fp)
    accuracy = (tp+tn)/number_datapoints
    return accuracy, sensitivity, specificity,precision




acc, sens, spec, prec = acc_sens_spec_prec(solution=solution, pos_represent=2, neg_represent=1)
print(acc_sens_spec_prec(solution=solution, pos_represent=2, neg_represent=1))

16281
16281
